In [1]:
import numpy as np #version 1.18.5
import pandas as pd #version 1.0.5
import json #version 2.0.9
import plotly.express as px #version 4.11.0

In [2]:
import dash #version 1.16.3
import dash_core_components as dcc #version 1.12.1 (pip install dash --upgrade)
import dash_html_components as html #version 1.1.1
from dash.dependencies import Input,Output

In [3]:
#the json
geo=json.load(open("brazil-states.geojson"))

#the data
df=pd.read_csv("amazon.csv",encoding="latin1")
df.drop(["date"],axis=1,inplace=True)
df.rename(columns={"year":"Year","state":"State","month":"Month","number":"Number"},inplace=True)

month_mapper={"Janeiro":"January","Fevereiro":"February","Março":"March","Abril":"April","Maio":"May","Junho":"June","Julho":"July","Agosto":"August","Setembro":"September","Outubro":"October","Novembro":"November","Dezembro":"December"}
df["Month"]=df["Month"].map(month_mapper)

name_mapper={"Acre":"Acre","Alagoas":"Alagoas","Amazonas":"Amazonas","Amapa":"Amapá","Bahia":"Bahia","Ceara":"Ceará","Distrito Federal":"Distrito Federal","Espirito Santo":"Espírito Santo","Goias":"Goiás","Maranhao":"Maranhão","Mato Grosso":"Mato Grosso","Minas Gerais":"Minas Gerais","Pará":"Pará","Paraiba":"Paraíba","Pernambuco":"Pernambuco","Piau":"Piauí","Rio":"Rio de Janeiro","Rondonia":"Rondônia","Roraima":"Roraima","Santa Catarina":"Santa Catarina","Sao Paulo":"São Paulo","Sergipe":"Sergipe","Tocantins":"Tocantins"}
df["State"]=df["State"].map(name_mapper)

state_codes=pd.read_csv("brazil-state-codes.csv")
state_codes=state_codes[["subdivision","name"]]
df=df.merge(state_codes,how="left",left_on="State",right_on="name")
df.rename(columns={"subdivision":"State_Code"},inplace=True)
df.drop(["name"],axis=1,inplace=True)

#the map data
def map_df_generator(year):
    number_i=[]
    for j in df["State"].unique():
        number_i.append(df["Number"].loc[(df["Year"]==year)&(df["State"]==j)].sum())
    df_i=pd.DataFrame({"Year":year,"State":df["State"].unique(),"State_Code":df["State_Code"].unique(),"Number":number_i})
    return df_i

df_map=pd.concat([map_df_generator(1998),map_df_generator(1999),map_df_generator(2000),map_df_generator(2001),map_df_generator(2002),map_df_generator(2003),map_df_generator(2004),map_df_generator(2005),map_df_generator(2006),map_df_generator(2007),map_df_generator(2008),map_df_generator(2009),map_df_generator(2010),map_df_generator(2011),map_df_generator(2012),map_df_generator(2013),map_df_generator(2014),map_df_generator(2015),map_df_generator(2016),map_df_generator(2017)])
df_map.reset_index(inplace=True)
df_map.drop(["index"],axis=1,inplace=True)

In [ ]:
#the text
intro_text="Brazil is home to lush rainforests which is vital for keeping our planet healthy. Unfortunately deforestation for agriculture and logging purposes has already dwindled away 20% of the Amazon rainforest. The land is usually cleared by burning which can get out of control and develop into wildfires.",html.Br(),html.Br(),"Select a state from the dropdown menu and a year from the slider to view the data."

#the dropdown
dropdown_options=[]

state_list=df["State"].unique()

for i in state_list:
    dropdown_options.append({"label":i,"value":i})

#the slider
slider_markers={1998:{"label":"1998"},2004:{"label":"2004"},2011:{"label":"2011"},2017:{"label":"2017"}}

In [ ]:
#figure_map
figure_map=px.choropleth_mapbox(df_map, geojson=geo,featureidkey="properties.sigla",locations="State_Code",color="Number",zoom=3,center={"lat":-13.017113,"lon":-51.074481},mapbox_style="carto-positron",color_continuous_scale="sunset",labels={"Number":"Number of Fires"},hover_name="State",hover_data={"State_Code":False,"Year":True,"Number":True},template="plotly_white")
figure_map.update_layout(margin={"t":0,"r":0,"b":0,"l":0})
figure_map.update_layout({"plot_bgcolor":"rgba(0,0,0,0)","paper_bgcolor":"rgba(0,0,0,0)"})

In [ ]:
#figure_year
figure_year=px.bar(df,x="Year",y="Number",color="State",labels={"Number":"Number of Fires"},hover_name="State",hover_data={"State":False,"Year":False,"Month":True,"Number":True},template="plotly_white")
figure_year.update_xaxes(tick0=1998,dtick=1)
figure_year.update_layout(margin={"t":10,"r":0,"b":0,"l":80})
figure_year.update_layout({"plot_bgcolor":"rgba(0,0,0,0)","paper_bgcolor":"rgba(0,0,0,0)"})
figure_year.update_layout(showlegend=False)

In [ ]:
#figure_month
figure_month=px.line(df,x="Month",y="Number",color="State",labels={"Number":"Number of Fires"},hover_name="State",hover_data={"State":False,"Month":False,"Year":True,"Number":True},template="plotly_white")
figure_month.update_layout(margin={"t":10,"r":0,"b":0,"l":80})
figure_month.update_layout({"plot_bgcolor":"rgba(0,0,0,0)","paper_bgcolor":"rgba(0,0,0,0)"})
figure_month.update_layout(showlegend=False)

In [ ]:
#original function
def update_figures(selected_year,selected_state):
        dff=df.copy()
        dfmap=dff[dff["Year"]==selected_year]
        dfyear=dff[dff["State"]==selected_state]
        dfmonth=dff[(dff["Year"]==selected_year)&(dff["State"]==selected_state)]

        figure_map=px.choropleth_mapbox(dfmap, geojson=geo,featureidkey="properties.sigla",locations="State_Code",color="Number",zoom=2.5,center={"lat":-13.017113,"lon":-51.074481},mapbox_style="carto-positron",color_continuous_scale="sunset",labels={"Number":"Number of Fires ('000s)"},hover_name="State",hover_data={"State_Code":False,"Year":True,"Number":True},template="plotly_white")
        figure_map.update_layout(margin={"t":0,"r":0,"b":0,"l":0})
        figure_map.update_layout({"plot_bgcolor":"rgba(0,0,0,0)","paper_bgcolor":"rgba(0,0,0,0)"})

        figure_year=px.bar(dfyear,x="Year",y="Number",labels={"Number":"Number of Fires ('000s)"},hover_name="State",hover_data={"State":False,"Year":True,"Month":True,"Number":True},color_discrete_sequence=["tomato"],template="plotly_white")
        figure_year.update_xaxes(tick0=1998,dtick=1)
        figure_year.update_layout(margin={"t":10,"r":0,"b":0,"l":80})
        figure_year.update_layout({"plot_bgcolor":"rgba(0,0,0,0)","paper_bgcolor":"rgba(0,0,0,0)"})
        figure_year.update_layout(showlegend=False)

        figure_month=px.line(dfmonth,x="Month",y="Number",labels={"Number":"Number of Fires ('000s)"},hover_data={"State":True,"Month":False,"Year":True,"Number":True},color_discrete_sequence=["tomato"],template="plotly_white")
        figure_month.update_layout(margin={"t":10,"r":0,"b":0,"l":80})
        figure_month.update_layout({"plot_bgcolor":"rgba(0,0,0,0)","paper_bgcolor":"rgba(0,0,0,0)"})
        figure_month.update_layout(showlegend=False)

        return figure_map,figure_year,figure_month

In [5]:
df_map[df_map["Year"]==1998]

,Year,State,State_Code,Number
0,1998,Acre,AC,730.000
1,1998,Alagoas,AL,86.000
2,1998,Amapá,AP,278.000
3,1998,Amazonas,AM,946.000
4,1998,Bahia,BA,1224.687
5,1998,Ceará,CE,1612.014
6,1998,Distrito Federal,DF,103.000
7,1998,Espírito Santo,ES,218.000
8,1998,Goiás,GO,750.119
9,1998,Maranhão,MA,365.884
